In [1]:
import numpy as np
import os
from osgeo import gdal
gdal.UseExceptions()

In [ ]:
# a = np.loadtxt('nlcd_nodevt_SF_bay_AOI_2005-2014_lrtb.csv', dtype='int')

In [2]:
fileroot = "../data/flickr/nlcd_grid_pud/"
filelist = ['nlcd_nodevt_utm_SF_bay_AOI_2005-2014_lrbt.csv', \
            'nlcd_nodevt_utm_SF_bay_AOI_2015_lrbt.csv', \
            'nlcd_nodevt_utm_SF_bay_AOI_2016_lrbt.csv', \
            'nlcd_nodevt_utm_SF_bay_AOI_2017_lrbt.csv']

In [ ]:
# fileroot = "../data/flickr/nlcd_grid_pud/scratch"
# filelist = ['a.csv', \
#             'b.csv', \
#             'c.csv', \
#             'd.csv']
# totals = np.zeros(shape=(10,), dtype='int')

In [3]:
! wc -l ../data/flickr/nlcd_grid_pud/nlcd_nodevt_utm_SF_bay_AOI_2005-2014_lrbt.csv

116888070 ../data/flickr/nlcd_grid_pud/nlcd_nodevt_utm_SF_bay_AOI_2005-2014_lrbt.csv


In [4]:
totals = np.zeros(shape=(116888070,), dtype='int')  # from line count of one input file

In [5]:
f1 = open(os.path.join(fileroot, filelist[0]),'rt')
f2 = open(os.path.join(fileroot, filelist[1]),'rt')
f3 = open(os.path.join(fileroot, filelist[2]),'rt')
f4 = open(os.path.join(fileroot, filelist[3]),'rt')

i = 0
for a, b, c, d in zip(f1, f2, f3, f4):
    x = np.array([a.rstrip('\n'), b.rstrip('\n'), c.rstrip('\n'), d.rstrip('\n')])
    x = x.astype(np.int)
    totals[i] = np.sum(x)
    i += 1
    if (i % 1000000==0):
        print(i)

np.savetxt(os.path.join(fileroot, 'nlcd_sfbay_2005_2017_totalpud.txt'), totals, fmt="%i")

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
68000000
69000000
70000000
71000000
72000000
73000000
74000000
75000000
76000000
77000000
78000000
79000000
80000000
81000000
82000000
83000000
84000000
85000000
86000000
87000000
88000000
89000000
90000000
91000000
92000000
93000000
94000000
95000000
96000000
97000000
98000000
99000000
100000000
101000000
102000000
103000000
104000000
105000000
106000000
107000000
108000000
109000000
110000000
11100000

In [ ]:
# totals = np.loadtxt(os.path.join(fileroot, 'nlcd_sfbay_2005_2017_totalpud.txt'), dtype='int')

In [6]:
ds = gdal.Open('/home/dmf/Recdev/data/SF_Bay/data/bcdc_othernaturalareas/NaturalAreas_ForDave/nlcd_nodevt_utm.tif')

In [7]:
ds.RasterXSize*ds.RasterYSize == len(totals)

True

In [8]:
# how to call reshape in order to 
# fill each row from left to right, starting with bottom row and moving up
# test = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
# np.flip(np.reshape(test, (4, 3)), 0) # default order of reshape works, then flip along 0 axis
totalmatrix = np.flip(np.reshape(totals, (ds.RasterYSize, ds.RasterXSize)), 0)

In [9]:
driver = gdal.GetDriverByName('GTiff')

In [10]:
new_ds = driver.Create(os.path.join(fileroot, 'pud_nlcdgrid.tif'),
    ds.RasterXSize, # x size
    ds.RasterYSize, # y size
    1, # number of bands
    gdal.GDT_Int16)

In [11]:
new_band = new_ds.GetRasterBand(1)
new_band.WriteArray(totalmatrix)
# flush data to disk, set the NoData value and calculate stats
new_band.FlushCache()
# new_band.SetNoDataValue(nodata)

# georeference the image and set the projection
new_ds.SetGeoTransform(ds.GetGeoTransform())
new_ds.SetProjection(ds.GetProjection())
del new_ds